In [16]:
import numpy as np
from random import random
from random import randint
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from time import time
from gensim.models import Word2Vec
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from scipy.sparse import identity, diags
from urllib.request import urlopen
import torch.nn.functional as F
from sklearn.metrics import log_loss, accuracy_score
import re
from gensim.parsing.preprocessing import remove_stopwords
from random import choice


#!pip install unidecode
from unidecode import unidecode

#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ghassenabdedayem/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ghassenabdedayem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:



def text_to_list(text):
    return unidecode(text).split(',')

def intersection(lst1, lst2): # a function that returns the number of common items of two lists and 1 or 0 if there are common. This function will be used in add_authors_to_pairs to add this features to the pairs.
    lst3 = [value for value in lst1 if value in lst2]
    is_common = 1 if len(lst3)>0 else 0
    return len(lst3), is_common


def save_subgraph_in_file(nbr_nodes, source_path='../input_data/edgelist.txt', destination_path='../input_data/small_edgelist.txt'):
    G = nx.read_edgelist(source_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    G = G.subgraph(range(nbr_nodes))
    nx.write_edgelist(G, path=destination_path, delimiter=',')
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph extracted from', source_path[source_path.rfind('/')+1:])
    G = nx.read_edgelist(destination_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph saved in', destination_path[destination_path.rfind('/')+1:])
    print(max(G.nodes))
    return



def read_train_val_graph(path='../input_data/edgelist.txt', val_ratio=0.1):
    #gets the data from the file on the distant server
    #G = nx.read_edgelist(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/edgelist.txt'), delimiter=',', create_using=nx.Graph(), nodetype=int)
    G = nx.read_edgelist(path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    nodes = list(G.nodes())
    n = G.number_of_nodes()
    m = G.number_of_edges()
    edges = list(G.edges())

    print('Number of nodes:', n, 'number of edges:', m,'in the Complete the set')

    node_to_idx = dict()
    for i, node in enumerate(nodes):
        node_to_idx[node] = i

    val_edges = list()
    G_train = G.copy()

    for edge in edges:
        if random() < val_ratio and edge[0] < n and edge[1] < n:
            val_edges.append(edge)
            G_train.remove_edge(edge[0], edge[1]) # We remove the val edges from the graph G

   
    #for edge in val_edges:
        

    n = G_train.number_of_nodes()
    m = G_train.number_of_edges()
    train_edges = list(G_train.edges())

    print('Number of nodes:', n, 'number of edges:', m, 'in the Training set')
    print('len(nodes)', len(nodes))

    y_val = [1]*len(val_edges)

    n_val_edges = len(val_edges)
    
    print('Creating random val_edges...')
    for i in range(n_val_edges):
        n1 = nodes[randint(0, n-1)]
        n2 = nodes[randint(0, n-1)]
        (n1, n2) = (min(n1, n2), max(n1, n2))
        while n2 >= n: #or (n1, n2) in train_edges:
            if (n1, n2) in train_edges:
                print((n1, n2), 'in train_edges:')
            n1 = nodes[randint(0, n-1)]
            n2 = nodes[randint(0, n-1)]
            (n1, n2) = (min(n1, n2), max(n1, n2))
        val_edges.append((n1, n2))

    y_val.extend([0]*(n_val_edges))
    
    ### From Giannis /!\
    val_indices = np.zeros((2,len(val_edges)))
    for i,edge in enumerate(val_edges):
        val_indices[0,i] = node_to_idx[edge[0]]
        val_indices[1,i] = node_to_idx[edge[1]]
    
    print('Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects')
    print('Loaded from', path[path.rfind('/')+1:], 'and with a training validation split ratio =', val_ratio)
    
    
    
    return G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx

def random_walk(G, node, walk_length):
    walk = [node]
  
    for i in range(walk_length-1):
        neibor_nodes = list(G.neighbors(walk[-1]))
        if len(neibor_nodes) > 0:
            next_node = choice(neibor_nodes)
            walk.append(next_node)
    walk = [node for node in walk] # in case the nodes are in string format, we don't need to cast into string, but if the nodes are in numeric or integer, we need this line to cast into string
    return walk


def generate_walks(G, num_walks, walk_length):
  # Runs "num_walks" random walks from each node, and returns a list of all random walk
    t = time()
    print('Start generating walks....')
    walks = list()  
    for i in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
        #print('walks : ', walks)
    print('Random walks generated in in {}s!'.format(round(time()-t)))
    return walks

def apply_word2vec_on_features(features, nodes, vector_size=128, window=5, min_count=0, sg=1, workers=8):
    t = time()
    print('Start applying Word2Vec...')
    wv_model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=workers)
    wv_model.build_vocab(features)
    wv_model.train(features, total_examples=wv_model.corpus_count, epochs=5) 
    print('Word2vec model trained on features in {} min!'.format(round((time()-t)/60)))
    features_np = []
    for node in nodes:
        features_np.append(wv_model.wv[node])

    features_np = np.array(features_np)
    print(features_np.shape, 'features numpy array created in {} min!'.format(round((time()-t)/60)))
    return features_np



def normalize_adjacency(A):
    n = A.shape[0]
    A = A + identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D_inv = diags(inv_degs)
    A_hat = D_inv.dot(A)
    return A_hat

def create_and_normalize_adjacency(G):
    adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph
    adj = normalize_adjacency(adj)
    print('Created a normalized adjancency matrix of shape', adj.shape)
    indices = np.array(adj.nonzero()) # Gets the positions of non zeros of adj into indices
    print('Created indices', indices.shape, 'with the positions of non zeros in adj matrix')
    return adj, indices

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
 

def add_authors_to_pairs (pairs, authors):
    #authors = pd.DataFrame(authors)
    try: 
        pairs = pairs.detach().cpu().numpy()
    except:
        pass
        

    pairs_df = pd.DataFrame(np.transpose(pairs)).rename(columns={0: "paper_1", 1: "paper_2"})
    #pairs = torch.tensor(pairs).to(device)
    pairs_df = pairs_df.merge(authors, left_on='paper_1', right_on='paper_id', how='left').rename(columns={'authors': "authors_1"})
    pairs_df = pairs_df.merge(authors, left_on='paper_2', right_on='paper_id', how='left').rename(columns={'authors': "authors_2"})
    pairs_df.drop(['paper_id_x', 'paper_id_y'], axis=1, inplace=True)

    pairs_df['nb_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[0], axis=1)
    pairs_df['is_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[1], axis=1)

    #pairs_tensor = torch.LongTensor(np.transpose(pairs_df[["paper_1", "paper_2", 'is_common_author', 'nb_common_author']].values.tolist())).to(device)
    
    return np.transpose(pairs_df[["paper_1", "paper_2", 'is_common_author', 'nb_common_author']].values.tolist())





In [3]:
def read_and_clean_abstracts (sample_length=-1, abstracts_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/abstracts.txt'):
    t = time()
    abstracts = dict()
    f = urlopen(abstracts_path)
    
    for i, line in enumerate(f):
        if i == sample_length:
            break
        node, abstract = str(line).lower().split('|--|')
        abstract = remove_stopwords(abstract)
        abstract = re.sub(r"[,.;@#?!&$()-]", " ", abstract)

        for word in abstract.split()[:-1]:
            #abstract = abstract.replace(word, stemmer.stem(word))
            abstract = abstract.replace(word, lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word), pos='s'), pos='a'), pos='n'), pos='v'), pos='r'))
        
        node = re.sub("[^0-9]", "", node)
        abstracts[int(node)] = abstract
        
    print('Text loaded and cleaned in {:.0f} sec'.format(time()-t))
    return abstracts

In [4]:
class Vocabulary:

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.words_list = []
        self.num_words = 0
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.words_list.append(word)
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence.split()[:-1]:
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

    def words(self):
        return self.words_list



In [5]:
def doc_counter (documents, word): #a function that return the number of documents containing a word
    counter = 0
    for i in documents:
        if word in documents[i]:
            counter += 1
    return counter


In [6]:
n = -1 #length of the sample to develop and test the pipeline

#takes three minutes to process all the abstracts
abstracts = read_and_clean_abstracts(sample_length=n)  #149s #194s
abstracts_list_words = {i: abstracts[i].split()[:-1] for i in range(len(abstracts))}

#we create a vacabulary of words and sentences (abstracts)
#we take only a sample of 3 abstracts (i=2) to explore the approach
voc = Vocabulary('abstracts') 
for i in range(len(abstracts)):
    voc.add_sentence(abstracts[i])


Text loaded and cleaned in 176 sec


In [7]:
#convert words into digits using the vocabulary
abstracts_idx = {}

for i in range(len(abstracts_list_words)):
    abstracts_idx[i] = list()
    for word in abstracts_list_words[i]:
        word_idx = voc.to_index(word)
        abstracts_idx[i].append(word_idx)


In [9]:
#create the tf-idf matrix
#for a better performance we replaced the words by their indexes in the vocab
t = time()
tf_idf = {}
N = len(abstracts_idx)
for i in range(N):
    document = abstracts_idx[i]
    for word_idx in document:
        t = document.count(word_idx)
        d = len(document)
        if t == 0:
            tf = 0
        else:
            tf = 1 + np.log(t/d) #we need to change it into log tf
        df = doc_counter(abstracts_idx, word_idx)
        idf = np.log10(N/df)
        tf_idf[i, voc.to_word(word_idx)] = tf * idf
print('tf-idf matrix generated in {:.0f} sec'.format(time()-t))

KeyboardInterrupt: 

In [ ]:
break

In [ ]:
tf_idf = {}


for i in range(len(abstracts)):
    tokens = processed_text[i]
    counter = Counter(tokens + processed_title[i])
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log(N/(df+1))
        tf_idf[doc, token] = tf*idf

In [ ]:
sample_list = abstracts[0].split()[:-1]
my_dict = {i:sample_list.count(i) for i in sample_list}
my_dict

In [ ]:
break

In [ ]:
embedded_mean_abstracts = dict()

for node, abstract in enumerate(abstracts):
    embedded_mean_abstracts[node] = mean_w2v(abstracts[node])


In [ ]:
G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx = read_train_val_graph(val_ratio=0.1)


In [ ]:
list_abstracts = list()  
for node in G_train.nodes():
    list_abstracts.append(embedded_mean_abstracts[node])

print(np.shape(list_abstracts))

In [ ]:
adj, indices = create_and_normalize_adjacency(G_train)
features_np = list_abstracts

In [19]:
walks

[[0, 2, 55, 2, 26, 2, 36, 52, 32, 591, 706, 407, 52, 407, 678],
 [1, 24, 1, 20, 23, 1, 12, 10, 13, 10, 18, 10, 14, 10, 16],
 [2, 0, 2, 42, 723, 407, 767, 407, 614, 407, 556, 758, 556, 407, 750],
 [3, 1, 13, 1, 5, 1, 20, 1, 13, 1, 20, 1, 6, 21, 23],
 [5, 1, 5, 1, 15, 1, 19, 1, 6, 16, 1, 5, 1, 10, 13],
 [6, 21, 6, 21, 6, 1, 7, 8, 7, 8, 7, 8, 7, 8, 7],
 [7, 1, 10, 14, 1, 3, 1, 10, 18, 10, 13, 1, 21, 9, 21],
 [9, 1, 9, 1, 16, 6, 1, 12, 1, 6, 16, 10, 16, 6, 1],
 [10, 13, 10, 16, 1, 11, 1, 20, 1, 3, 1, 16, 10, 16, 1],
 [11, 1, 7, 8, 7, 8, 7, 1, 11, 1, 24, 1, 16, 6, 21],
 [12, 1, 10, 12, 1, 10, 1, 12, 10, 12, 10, 14, 10, 13, 1],
 [13, 10, 16, 1, 16, 1, 14, 10, 12, 1, 23, 20, 23, 20, 23],
 [14, 10, 14, 1, 23, 20, 1, 21, 23, 20, 1, 15, 1, 6, 1],
 [15, 1, 7, 1, 24, 1, 0, 1, 5, 1, 9, 21, 23, 1, 5],
 [16, 10, 12, 1, 14, 1, 13, 1, 5, 1, 9, 21, 9, 21, 9],
 [17],
 [19, 1, 3, 1, 6, 1, 22, 1, 5, 1, 12, 10, 14, 1, 21],
 [20, 23, 20, 23, 21, 9, 1, 3, 1, 0, 1, 13, 1, 5, 1],
 [21, 9, 1, 24, 1, 6, 1, 20, 1,

In [ ]:
break

In [17]:
path = '../input_data/small_edgelist.txt' #not used
G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx = read_train_val_graph(val_ratio=0.1, path=path)
walks = generate_walks(G=G_train, num_walks=10, walk_length=15)
walks_wv = apply_word2vec_on_features(features=walks, nodes=nodes, vector_size=16)
adj, indices = create_and_normalize_adjacency(G_train)

features_np = walks_wv

Number of nodes: 980 number of edges: 2405 in the Complete the set
Number of nodes: 980 number of edges: 2170 in the Training set
len(nodes) 980
Creating random val_edges...
Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects
Loaded from small_edgelist.txt and with a training validation split ratio = 0.1
Start generating walks....
Random walks generated in in 0s!
Start applying Word2Vec...
Word2vec model trained on features in 0 min!
(980, 16) features numpy array created in 0 min!
Created a normalized adjancency matrix of shape (980, 980)
Created indices (2, 5320) with the positions of non zeros in adj matrix


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_46545/4097758665.py:140: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph


In [ ]:
authors = pd.read_csv(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/authors.txt'), sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
authors = authors[["paper_id", "authors"]]
authors = authors[authors['paper_id'] <= max(G.nodes())]
authors.head()

In [ ]:
# Create class labels
y = np.zeros(2*indices.shape[1])
y[:indices.shape[1]] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.

# Transforms the numpy matrices/vectors to torch tensors.
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
if type(adj) != torch.Tensor:
    adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
indices = torch.LongTensor(indices).to(device)
val_indices = torch.LongTensor(val_indices).to(device)
y_val = torch.LongTensor(y_val).to(device)

del (G, G_train, train_edges, val_edges, nodes, abstracts, embedded_mean_abstracts)



In [ ]:
class GNN(nn.Module):
    def __init__(self, n_feat, n_hidden, n_class, sub_class, dropout):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(n_feat, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.double_fc3 = nn.Linear((3*n_hidden), n_hidden)
        self.fc4 = nn.Linear(n_hidden, sub_class)
        self.fc5 = nn.Linear(sub_class, n_class)        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
        

    def forward(self, x_in, adj, pairs):
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.mm(adj, h1))
        z1 = self.dropout(z1)
        del(x_in, h1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.mm(adj, h2))
        z2 = self.dropout(z2)
        del(h2, z1, adj)

        x = z2[pairs[0]] - z2[pairs[1]] # embedded features (z2) of node 0 - embedded features of node 1
        
        x = pairs[3][:, None] * x
        x = x.to(device)
        x1 = z2[pairs[0]]#.to(device)
        x2 = z2[pairs[1]]#.to(device)
        del(pairs)
        x = torch.cat((x, x1, x2), dim=1)
        del(x1, x2)
        

        x = self.relu(self.double_fc3(x))        
        x = self.dropout(x)
        
        x = self.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.fc5(x)

        return F.log_softmax(x, dim=1)




In [ ]:
def early_stopping(loss_train, list_loss_train, loss_val, list_loss_val, window=10, tolerance=0.01):
    list_loss_val = list(list_loss_val)[-window:]
    list_loss_train = list(list_loss_train)[-window:]
    if (len(list_loss_val) == window and loss_val > (sum(list_loss_val)/len(list_loss_val)) and loss_train + tolerance < loss_val) or (len(list_loss_train) == window and loss_train > (sum(list_loss_train)/len(list_loss_train))):
        print('train: {:.5f} val: {:.5f} mean val: {:.5f}'.format(loss_train, loss_val, (sum(list_loss_val)/len(list_loss_val))))
        return True
    return False
    

    
def train_model(model, learning_rate, features, authors, adj, indices, y, val_indices, y_val, epochs, run_number):
    # Train model
    start_time = time()
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training
    try: os.mkdir('./outputs')
    except: pass
    print('Preparing the data for training...')        
    val_indices = add_authors_to_pairs(val_indices, authors) #we add the authors to val_pairs
    indices = add_authors_to_pairs(indices, authors) #we add the authors to indices
    rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)# We take random indices each time we run an epoch
    rand_indices = add_authors_to_pairs(rand_indices, authors)
    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices. 
    pairs = torch.tensor(pairs).to(device)
    del(authors, indices, rand_indices)

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []
    
    features = features.to(device)
    adj = adj.to(device)
    pairs = pairs.to(device)
    
    halving_lr = 0 # counter of the number of halving lr
    print('Start training...')
    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        
        model.train()

        output = model(features, adj, pairs).to(device) # we run the model that gives the output.
        loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
        acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
        loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, adj, val_indices).to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        
        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
        if epoch % 20 == 0:
            model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
            torch.save(model.state_dict(), model_path)
            
        early = early_stopping(loss_train.item(), list_loss_train, loss_val.item(), list_loss_val, window=10)        
        if early:
            halving_lr += 1
            if halving_lr > 10:
                break
            list_loss_val=[]
            learning_rate = learning_rate/2
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            print('Deviding the learning rate by 2. New learning rate: {:.6f}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model



In [ ]:
n_hidden = 64
dropout_rate = 0.2
sub_class = 8
n_class = 2
n_features = features.shape[1]

# Creates the model
model = GNN(n_features, n_hidden, n_class, sub_class, dropout_rate).to(device)

In [ ]:
features.get_device()

In [ ]:
torch.cuda.empty_cache()

#import gc
#del variables
#gc.collect()

epochs = 1000
run_number = randint(0, 1000)


trained_model = train_model(model, 0.02, features, authors, adj, indices, y, val_indices, y_val, epochs, run_number)


In [ ]:
np.transpose(node_pairs)

In [ ]:
from datetime import datetime

test_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/test.txt'
node_pairs = list()
f = urlopen(test_path)

for line in f:
    t = str(line).split(',')
    t[0] = int(re.sub("[^0-9]", "", t[0]))
    t[1] = int(re.sub("[^0-9]", "", t[1]))
    node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

node_pairs = np.transpose(node_pairs)
node_pairs = add_authors_to_pairs(node_pairs, authors)
node_pairs = torch.LongTensor(node_pairs).to(device)

test_output = model(features, adj, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])

today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)
model_nb = 1

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
node_pairs = torch.LongTensor(node_pairs).to(device)

test_output = model(features, adj, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])
    

    
today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
epochs = 1000

trained_model = train_model(model, 0.01, features, authors, adj, indices, y, torch.tensor(val_indices).to(device), torch.tensor(y_val).to(device), epochs)


In [ ]:
print(type(features), type(adj), type(indices), type(y))
print(features.get_device(), adj.get_device(), indices.get_device(), y.get_device())

#torch.tensor(np.array(authors)).to(device).get_device()
authors
print(type(torch.tensor(val_indices)))
print(torch.tensor(val_indices).to(device).get_device())

type(y_val)

####### randindicies to check on which device

In [ ]:
import torch
print (torch.__version__)
!pip install torchvision==0.14.0
!pip install torchtext==0.14.0
!pip install torchaudio==0.13.0
!pip install torch==1.13.0
import torch
print (torch.__version__)